In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("taxi-fare-prediction")\
                    .config("spark.executor.memory", MAX_MEMORY)\
                    .config("spark.driver.memory", MAX_MEMORY)\
                    .getOrCreate()

22/07/31 22:19:54 WARN Utils: Your hostname, singyeongdeog-ui-Macmini.local resolves to a loopback address: 127.0.0.1; using 222.98.22.103 instead (on interface en0)
22/07/31 22:19:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/31 22:19:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
trip_files = "/Users/singyeongdeog/Documents/github_code/data-engineering/01-spark/data/yellow/*"

In [4]:
trips_df = spark.read.csv(f"file:///{trip_files}", inferSchema=True, header=True)

In [5]:
trips_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [6]:
trips_df.createOrReplaceTempView("trips")

In [7]:
query = """
SELECT
    trip_distance,
    total_amount
FROM
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_dropoff_datetime) < '2021-08-01'
"""

In [9]:
data_df = spark.sql(query)
data_df.createOrReplaceTempView("data")

In [10]:
data_df.show()

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|          2.1|        11.8|
|          0.2|         4.3|
|         14.7|       51.95|
|         10.6|       36.35|
|         4.94|       24.36|
|          1.6|       14.15|
|          4.1|        17.3|
|          5.7|        21.8|
|          9.1|        28.8|
|          2.7|       18.95|
|         6.11|        24.3|
|         1.21|       10.79|
|          7.4|       33.92|
|         1.01|        10.3|
|         0.73|       12.09|
|         1.17|       12.36|
|         0.78|        9.96|
|         1.66|        12.3|
|         0.93|         9.3|
|         1.16|       11.84|
+-------------+------------+
only showing top 20 rows



In [11]:
data_df.describe().show()

+-------+------------------+------------------+
|summary|     trip_distance|      total_amount|
+-------+------------------+------------------+
|  count|          13125324|          13125324|
|   mean| 2.881893450401659|17.972483560192323|
| stddev|3.8200814810687933|12.975154876440737|
|    min|              0.01|              0.01|
|    max|             475.5|            4973.3|
+-------+------------------+------------------+



In [12]:
train_df, test_df = data_df.randomSplit([0.8,0.2], seed=1)

In [13]:
print(train_df.count())
print(test_df.count())

10499540


2625784


In [14]:
from pyspark.ml.feature import VectorAssembler

In [22]:
vassembler = VectorAssembler(inputCols=["trip_distance"], outputCol="features")

In [23]:
vtrain_df = vassembler.transform(train_df)

In [24]:
vtrain_df.show()

+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+
only showing top 20 rows



In [25]:
from pyspark.ml.regression import LinearRegression

In [26]:
lr = LinearRegression(
    maxIter = 50,
    labelCol = "total_amount",
    featuresCol = "features"
)

In [27]:
model = lr.fit(vtrain_df)

22/07/31 22:32:33 WARN Instrumentation: [cd080a2f] regParam is zero, which might cause numerical instability and overfitting.


22/07/31 22:32:40 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/07/31 22:32:40 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


22/07/31 22:33:12 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [28]:
vtest_df = vassembler.transform(test_df)

In [29]:
prediction = model.transform(vtest_df)

In [30]:
prediction.show()

+-------------+------------+--------+-----------------+
|trip_distance|total_amount|features|       prediction|
+-------------+------------+--------+-----------------+
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.3|  [0.01]|9.450572628399861|
|         0.01|         3.8|  [0.01]|9.450572628

In [31]:
model.summary.rootMeanSquaredError

6.32310203992383

In [32]:
model.summary.r2

0.7634924733680508

In [34]:
from pyspark.sql.types import DoubleType
distance_list = [1.1, 5.5, 10.5, 30.0]
distance_df = spark.createDataFrame(distance_list, DoubleType()).toDF("trip_distance")

In [35]:
distance_df.show()

+-------------+
|trip_distance|
+-------------+
|          1.1|
|          5.5|
|         10.5|
|         30.0|
+-------------+



In [36]:
vdistance_df = vassembler.transform(distance_df)

In [37]:
vdistance_df.show()

+-------------+--------+
|trip_distance|features|
+-------------+--------+
|          1.1|   [1.1]|
|          5.5|   [5.5]|
|         10.5|  [10.5]|
|         30.0|  [30.0]|
+-------------+--------+



In [38]:
model.transform(vdistance_df).show()

+-------------+--------+------------------+
|trip_distance|features|        prediction|
+-------------+--------+------------------+
|          1.1|   [1.1]|12.684655849989994|
|          5.5|   [5.5]|  25.7396706894364|
|         10.5|  [10.5]| 40.57491482517095|
|         30.0|  [30.0]|  98.4323669545357|
+-------------+--------+------------------+

